# Optimization Model - Hybrid Energy Generation & Storage: 1 Battery-1 Genset .
=========================================================================================================================
$ 
\\
$

$ 
\\
$

### Installing PuLP Modeler.

In [32]:
%pip install pulp

Note: you may need to restart the kernel to use updated packages.


### Testing PuLP packages.

In [ ]:
import pulp
pulp.pulpTestAll()

### Importing Libraries.

In [34]:
import numpy as np
from collections import Counter
import math
from pulp import *

### Introducing problem Parameters.

In [35]:
Q_max = 250                                                                         # Maximal capacity stored on battery
Q_0 = 0.2*Q_max                                                                     # Inital capacity stored on battery
P_A_max = 800                                                                       # Max output from gen set A
dt = 1                                                                              # Time step in simulation dt=1min
t_max = 60                                                                          # Time span for simulation = t_max hours
t = np.atleast_2d(np.arange(0,t_max,dt)).T.conj()
n = len(t)                                                                          # Number of time steps
m = 5                                                                               # Number of Gensets on the ship 
a_j = 1.5                                                                           # Slope for the linear model of genset j consumption 
fc_j_offset = 500                                                                   # Genset j fuel consumption when no power gnerated (p=0) in g/h 
y_k_j = 1                                                                           # Genset j pointer at time step k (if genset j is used then = 1 otherwise = 0)
y_k_j_1 = 0                                                                         # Henset j pointer at time step k-1 (if genset j is used then = 1 otherwise = 0)
K_j_start = dict.fromkeys([x for x in range(60)],0)
K_j_start[0]=0.1                                                                    # Additional fuel consumption when starting genset j (kg)

                                                                                    
import sys, os
sys.path.append(os.path.abspath(os.path.join(r'wpath', r'ppath')))                  # Bringing the packages onto the path


from Load_window import Load_window                                                 # Now we import modules of Load_window_function
L = Load_window (200,200,10,400,400,10,600,600,10,600,200,30,0,10,20,30)


from Dictionaries_add import merge_dictionaries_1_2                                 # Now we import modules of Dictionaries_add



### Creating the problem variable.

In [36]:
prob = LpProblem('Energy_Opt',LpMinimize)

### Setting-up decision Variables.

In [37]:
steps = np.arange(0,n)
V_steps =  range(0, steps.shape[0]) 

Q_bat = pulp.LpVariable.dicts("Q_bat", V_steps, lowBound=0.2*Q_max, upBound=Q_max, cat = LpContinuous)
P_bat = pulp.LpVariable.dicts("P_bat", V_steps, lowBound=0, upBound=None, cat = LpContinuous)
P_A = pulp.LpVariable.dicts("P_A", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpInteger)
P_A_load = pulp.LpVariable.dicts("P_A_load", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpInteger)
P_A_bat = pulp.LpVariable.dicts("P_A_bat", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpContinuous)
SFOC_A = pulp.LpVariable.dicts("SFOC_A", V_steps, lowBound=None, upBound=None, cat = LpInteger)
Z_k_j = pulp.LpVariable.dicts("Z_k_j", V_steps, lowBound=0, upBound=1, cat = LpBinary)

### Setting-up the objective function.

In [38]:
for j in range(1,m):
  for k in range(0,n):
    FC_k_j = {key: P_A[key]*a_j + fc_j_offset * y_k_j  for key in P_A}   # linear model for fuel consumption of genset j at time step k
    FC = sum(sum(FC_k_j[k] * dt/1000 * k for k in range(0,n) )*j for j in range(1,m)) # sum of the fuel oil comsumption for all gensets over all time steps.
    L_added_cost = sum(Z_k_j[j] * j for j in range(1,m)) * sum(K_j_start[k] * k for k in range(0,n) ) # sum of all of the additional costs including starting costs. 
    prob += FC + L_added_cost # The minimize objective function.

c:\ProgramData\Anaconda3\lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


### Setting-up problem Constraints.

In [39]:

prob += L == merge_dictionaries_1_2(P_A_load, P_bat), "Load Requirements"                                               # Load requirements constraints
prob += P_A == merge_dictionaries_1_2(P_A_load, P_A_bat), "Power Split"                                                 # Power split


for x in P_A: 
  P_A_A = dict.fromkeys([x for x in range(60)])
  P_A_A[x] = P_A[x]*(-67/P_A_max) + 260
  prob += SFOC_A == P_A_A, "Specific Fuel Oil Consumption of Genset A"                                                  # Genset specific Fuel oil consumption 

for x in range(0,0):
  prob += Q_bat[0] == Q_0, "Initial Charge balance"                                                                     # Initial charge balance


Q_init = dict.fromkeys([x for x in range(60)], 0)
Q_k = dict.fromkeys([x for x in range(60)], 0)

for k in range(1,n,1):
  Q_k[k] = (P_A_bat[k] - P_bat[k])*dt

for i in range(0,n-1,1):
  Q_init[i] = Q_bat[i]


  prob += Q_bat == merge_dictionaries_1_2(Q_init,Q_k), "Charge balance at time step k"                                  # Charge balance 



### Solving the problem.

In [40]:
status = prob.solve()

### Optimization result.

In [41]:
LpStatus[status]

'Optimal'

### Printing constraints per each time step.

In [ ]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

### Calculating the optimized fuel comsumption.

In [43]:
print("Total optimized fuel comsumption of the trip :", value(prob.objective),'Kg')

Total optimized fuel comsumption of the trip : 13098.0 Kg
